In [1]:
import warnings

import numpy as np
import pandas as pd
import sklearn.datasets as datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import torch
import torch.nn as nn
import torch.optim as optimizers
from torch.utils.data import Dataset, DataLoader
import xgboost as xgb
import lightgbm as lgb
import optuna
import optuna.integration.lightgbm as lgbo

warnings.filterwarnings("ignore")

/usr/local/lib/python3.8/dist-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
# データの読み込み
# 例としてscikit-learnで提供されているカリフォルニアの住宅価格データを用いる
data = datasets.fetch_california_housing()

X = data["data"]
y = data["target"]

# 見やすいようにDataFrameにする
df = pd.DataFrame(np.hstack((X, y.reshape(-1, 1))),
                  columns=data.feature_names+["price"])
display(df)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


## XGBOOSTをOptunaで最適化する  
Optunaはベイズ最適化を用いてパラメータ最適化をするライブラリです．ランダムサーチやグリッドサーチよりも効率的にパラメータ最適化を行うことができます．

最適化の際には関数として一連の処理を定義しておく必要があります．

1. 最適化したいパラメータと，パラメータの取りうる値を定義します．パラメータの取りうる値はtrial.suggest_intやtrial.suggest_loguniformなどで指定します．  
2. 設定したパラメータをモデルに渡してモデルの学習と，評価を行います．  
3. 最小化，もしくは最大化したい値を関数の返り値として設定します．

最適化するパラメータ，最小化，もしくは最大化する目的関数を自由に設定することができるため，基本的にどのようなモデルにも適用できる，目的関数も自分で実装して最適化することができます．また今回は最適化対象のパラメータが整数や小数でしたが，質量変数を指定する場合はsuggest_categoricalを用いるなどできます．より詳細には[公式ページ](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.trial.Trial.html)がありますのでパラメータによって使用するものを変えるようにしてください．  

その後optuna.create_studyでインスタンスを作成します．その際の引き数に"direction"というものがあり，"maximize"を指定することで目的関数を最大化するように，"minimize"を指定することで最小化するようにパラメータを最適化するようにします．

インスタンスのoptimizeメソッドに最適化するための関数，イテレータ数を指定することで最適化が始まります．timeoutを指定することで一定時間がかかると処理を中断するように設定できます．

In [3]:
def objective(trial):
    # データを訓練用と検証用に分割する
    train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.2, random_state=6174)

    # 最適化したいパラメータと，パラメータがとる値の範囲を指定する
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "max_leaves": trial.suggest_int("max_leaves", 1, 30),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-4, 1e-1),
    }

    # モデルの訓練
    model = xgb.XGBRegressor(**params)
    model.fit(train_X, train_y)

    # モデルの評価
    pred = model.predict(val_X)
    score = mse(val_y, pred, squared=False)

    return score


study = optuna.create_study(direction="minimize")  # maximizeにすると最大化問題になる
study.optimize(objective, 30, timeout=600)

[I 2022-05-30 09:13:55,575] A new study created in memory with name: no-name-e01a8900-c95a-4b97-85ba-a6f5b3533c2d
[I 2022-05-30 09:13:57,357] Trial 0 finished with value: 0.4716889859796756 and parameters: {'n_estimators': 340, 'max_depth': 7, 'max_leaves': 12, 'learning_rate': 0.03399685126151244}. Best is trial 0 with value: 0.4716889859796756.
[I 2022-05-30 09:13:58,521] Trial 1 finished with value: 1.6117138939589892 and parameters: {'n_estimators': 152, 'max_depth': 9, 'max_leaves': 3, 'learning_rate': 0.0014762134546744993}. Best is trial 0 with value: 0.4716889859796756.
[I 2022-05-30 09:13:59,880] Trial 2 finished with value: 0.4943095012550664 and parameters: {'n_estimators': 298, 'max_depth': 6, 'max_leaves': 16, 'learning_rate': 0.021914669583863077}. Best is trial 0 with value: 0.4716889859796756.
[I 2022-05-30 09:14:01,548] Trial 3 finished with value: 1.653294909983414 and parameters: {'n_estimators': 310, 'max_depth': 7, 'max_leaves': 27, 'learning_rate': 0.0006356903636

パラメータの最適化が終わった後は，最も性能が良かったパラメータと，その時のスコアを表示することができます．studyのbest_params属性とbest_value属性にアクセスすることでこれらの値を得ることができます．

In [4]:
study.best_params

{'n_estimators': 267,
 'max_depth': 6,
 'max_leaves': 17,
 'learning_rate': 0.09518874548034575}

In [5]:
study.best_value

0.4634409527783798

簡単な使用方法は以上となります．最適化するパラメータや，評価手法（ホールドアウトを交差検証に変えるなど），目的関数を自由に変えることができるので基本的にどのようなタスク，モデルでも用いることができます．  

## PyTorchのモデルを最適化してみる  
あまり使用されているところ見たことがないのですが，PyTorchのモデルの隠れ層のニューロン数やドロップアウトの割合も最適化できます．  

In [6]:
# データセットの作成
# DataFrameから使用するデータを抽出し，1つずつ変えずだけのもの
class TestDataset(Dataset):
    def __init__(self, df, features, target):
        X = df[features].values
        y = df[target].values

        self.X = torch.tensor(X).float()
        self.y = torch.tensor(y).float()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [7]:
# 単純な3層ニューラルネット
class TestModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dr=0.3, device="cuda"):
        super().__init__()

        self.device = device
        self.hidden = nn.Linear(input_dim, hidden_dim)
        self.activation = nn.ReLU()
        self.dr = nn.Dropout(p=dr)
        self.out = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h = self.activation(self.hidden(x))
        dr_h = self.dr(h)
        y = self.out(dr_h)

        return y


# データの分割
train_df, val_df = train_test_split(df, test_size=0.2, random_state=6174)
features = df.columns[:-1]
target = df.columns[-1]

# ニューラルネット用にスケーリングする
sc = StandardScaler()
train_df[features] = sc.fit_transform(train_df[features])
val_df[features] = sc.transform(val_df[features])

# データローダーの作成
train_dataset = TestDataset(train_df, features, target)
val_dataset = TestDataset(val_df, features, target)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)

# モデル，損失関数，最適化手法の定義
model = TestModel(len(features), 32, 1).to("cuda")
criterion = nn.MSELoss()
optimizer = optimizers.Adam(model.parameters())


# 1イテレーション分の訓練
def train(batch_X, batch_y):
    model.train()
    batch_X, batch_y = batch_X.to("cuda"), batch_y.to("cuda")
    pred = model(batch_X)

    loss = torch.sqrt(criterion(pred, batch_y))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss


# 1イテレーション分の評価
def validation(batch_X, batch_y):
    model.eval()
    batch_X, batch_y = batch_X.to("cuda"), batch_y.to("cuda")
    pred = model(batch_X)

    loss = torch.sqrt(criterion(pred, batch_y))

    return loss

In [8]:
epochs = 10

# 10エポックで動作確認
for epoch in range(epochs):
    train_loss = 0.
    val_loss = 0.

    for batch_X, batch_y in train_dataloader:
        loss = train(batch_X, batch_y)
        train_loss += loss.item()

    train_loss /= len(train_dataloader)

    for batch_X, batch_y in val_dataloader:
        loss = validation(batch_X, batch_y)
        val_loss += loss.item()

    val_loss /= len(val_dataloader)

    print(f"Epoch {epoch+1} train loss: {train_loss:.4f} val_loss{val_loss:.4f}")

Epoch 1 train loss: 1.5460 val_loss0.9454
Epoch 2 train loss: 1.2571 val_loss0.9316
Epoch 3 train loss: 1.2179 val_loss0.9052
Epoch 4 train loss: 1.1988 val_loss0.8949
Epoch 5 train loss: 1.1894 val_loss0.8952
Epoch 6 train loss: 1.1859 val_loss0.8976
Epoch 7 train loss: 1.1811 val_loss0.9028
Epoch 8 train loss: 1.1768 val_loss0.8982
Epoch 9 train loss: 1.1733 val_loss0.9085
Epoch 10 train loss: 1.1727 val_loss0.8936


In [9]:
# 隠れ層のユニット数，ドロップアウトの割合を最適化の対象にする
def objective(trial):
    dr = trial.suggest_float("dr", 0.1, 0.6)
    hidden_dim = trial.suggest_categorical("hidden_dim", [2**(i+5) for i in range(5)])

    model = TestModel(len(features), hidden_dim, 1, dr=dr).to("cuda")
    criterion = nn.MSELoss()
    optimizer = optimizers.Adam(model.parameters())

    epochs = 30
    for epoch in range(epochs):
        train_loss = 0.
        val_loss = 0.

        for batch_X, batch_y in train_dataloader:
            loss = train(batch_X, batch_y)
            train_loss += loss.item()

        train_loss /= len(train_dataloader)

        for batch_X, batch_y in val_dataloader:
            loss = validation(batch_X, batch_y)
            val_loss += loss.item()

        val_loss /= len(val_dataloader)

    val_loss = 0.
    for batch_X, batch_y in val_dataloader:
        loss = validation(batch_X, batch_y)
        val_loss += loss.item()

    val_loss /= len(val_dataloader)

    return val_loss


study = optuna.create_study(direction="minimize")
study.optimize(objective, 10, timeout=600)

[I 2022-05-30 09:15:00,384] A new study created in memory with name: no-name-816200c0-0f49-4634-a6cf-ccce6723a9f7
[I 2022-05-30 09:15:33,649] Trial 0 finished with value: 0.9103587136076864 and parameters: {'dr': 0.3175997178494554, 'hidden_dim': 256}. Best is trial 0 with value: 0.9103587136076864.
[I 2022-05-30 09:16:06,729] Trial 1 finished with value: 0.8991878704102927 and parameters: {'dr': 0.12216728063565588, 'hidden_dim': 32}. Best is trial 1 with value: 0.8991878704102927.
[I 2022-05-30 09:16:39,628] Trial 2 finished with value: 0.908492523772541 and parameters: {'dr': 0.4359539731182841, 'hidden_dim': 512}. Best is trial 1 with value: 0.8991878704102927.
[I 2022-05-30 09:17:12,406] Trial 3 finished with value: 0.9050807212154532 and parameters: {'dr': 0.4957530594309756, 'hidden_dim': 256}. Best is trial 1 with value: 0.8991878704102927.
[I 2022-05-30 09:17:45,938] Trial 4 finished with value: 0.9038151067992052 and parameters: {'dr': 0.46029989631806445, 'hidden_dim': 128}.

## LightGBMの最適化
LightGBMの最適化の場合は，目的関数を定義する必要はなくlightgbm用の最適化関数が用意されています．これを用いることでLightGBMの場合は簡単に最適化することができます．  

In [17]:
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.2, random_state=6174)

lgb_train = lgb.Dataset(train_X, train_y)
lgb_val = lgb.Dataset(val_X, val_y)

# lightgbmの最低限設定したいパラメータ
params = {
    "objective": "regression",
    "metric": "mean_squared_error",
}

# モデルの学習
model = lgbo.train(params, lgb_train, valid_sets=lgb_val,
                   verbose_eval=False)

[I 2022-05-30 09:26:43,447] A new study created in memory with name: no-name-caab7f04-a1aa-4ff6-b163-087647e05c95
feature_fraction, val_score: inf:   0% 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000299 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1838
[LightGBM] [Info] Number of data points in the train set: 16512, number of used features: 8
[LightGBM] [Info] Start training from score 2.066357


feature_fraction, val_score: 0.195144:  14% 1/7 [00:00<00:02,  2.63it/s][I 2022-05-30 09:26:43,829] Trial 0 finished with value: 0.19514379281511343 and parameters: {'feature_fraction': 0.7}. Best is trial 0 with value: 0.19514379281511343.
feature_fraction, val_score: 0.195144:  14% 1/7 [00:00<00:02,  2.63it/s]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000277 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1838
[LightGBM] [Info] Number of data points in the train set: 16512, number of used features: 8
[LightGBM] [Info] Start training from score 2.066357


feature_fraction, val_score: 0.193633:  29% 2/7 [00:00<00:01,  2.53it/s][I 2022-05-30 09:26:44,234] Trial 1 finished with value: 0.19363338573467703 and parameters: {'feature_fraction': 0.5}. Best is trial 1 with value: 0.19363338573467703.
feature_fraction, val_score: 0.193633:  29% 2/7 [00:00<00:01,  2.53it/s]

...

min_data_in_leaf, val_score: 0.192888:  60% 3/5 [00:01<00:01,  1.93it/s][I 2022-05-30 09:27:20,888] Trial 65 finished with value: 0.1954040203758196 and parameters: {'min_child_samples': 5}. Best is trial 65 with value: 0.1954040203758196.
min_data_in_leaf, val_score: 0.192888:  60% 3/5 [00:01<00:01,  1.93it/s]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000282 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1838
[LightGBM] [Info] Number of data points in the train set: 16512, number of used features: 8
[LightGBM] [Info] Start training from score 2.066357


min_data_in_leaf, val_score: 0.192888:  80% 4/5 [00:02<00:00,  1.99it/s][I 2022-05-30 09:27:21,366] Trial 66 finished with value: 0.19572303008710432 and parameters: {'min_child_samples': 10}. Best is trial 65 with value: 0.1954040203758196.
min_data_in_leaf, val_score: 0.192888:  80% 4/5 [00:02<00:00,  1.99it/s]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000226 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1838
[LightGBM] [Info] Number of data points in the train set: 16512, number of used features: 8
[LightGBM] [Info] Start training from score 2.066357


min_data_in_leaf, val_score: 0.192888: 100% 5/5 [00:02<00:00,  2.02it/s][I 2022-05-30 09:27:21,850] Trial 67 finished with value: 0.19825709477162398 and parameters: {'min_child_samples': 25}. Best is trial 65 with value: 0.1954040203758196.
min_data_in_leaf, val_score: 0.192888: 100% 5/5 [00:02<00:00,  1.98it/s]


最適化後のモデルのパラメータはparams属性にアクセスすることで得ることができます．  

In [19]:
model.params

{'objective': 'regression',
 'metric': 'l2',
 'feature_pre_filter': False,
 'lambda_l1': 2.70997039083449e-06,
 'lambda_l2': 6.593468266917905e-05,
 'num_leaves': 42,
 'feature_fraction': 0.6,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 20,
 'num_iterations': 1000,
 'early_stopping_round': None}

また訓練したときのモデルをそのまま用いて予測をすることも可能です．

In [25]:
model.predict(val_X[0].reshape(1, -1))

array([4.39485911])

Optuna: https://optuna.readthedocs.io/en/stable/index.html  